In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Computation of collision events
Store:
* freq_col
* col_intens

In [ ]:
#Ship types (small to be consistent with frame's dimensions)
mass =[5,25,50,75,100] #t
v = 0.5 #m/s
col_intens = [0.5*m*(v**2)*1e-3 for m in mass] #MJ
freq_col = [1e-1*f for f in [1e-1, 1e-2, 5e-3, 1e-4, 1e-5]] #per year
print('lambdas: ', freq_col, ', energy_event: ', col_intens)

## Definition of geometry and mechanical parameters

In [ ]:
#Zayas frame braces 
z1,z2,z3,z4=0.702,3.048,3.048,1.524
x1,x2=0,3.048
xa,ya,dim_a = [x2,0.5*x2], [z1+z2+z3, z1+z2+0.5*z3], [0.102,0.002, 2.15526]
xb,yb,dim_b = [x1,0.5*x2], [z1+z2, z1+z2+0.5*z3], [0.102,0.002, 2.15526]
xc,yc,dim_c = [x1,0.5*x2], [z1+z2+z3, z1+z2+0.5*z3], [0.102,0.002, 2.15526]
xd,yd,dim_d = [x2,0.5*x2], [z1+z2, z1+z2+0.5*z3], [0.102,0.002, 2.15526]
xe,ye,dim_e = [x2,0.5*x2], [z1+z2, z1+0.5*z2], [0.127,0.003, 2.15526]
xf,yf,dim_f = [x1,0.5*x2], [z1, z1+0.5*z2], [0.127,0.003, 2.15526]
xg,yg,dim_g = [x1,0.5*x2], [z1+z2, z1+0.5*z2], [0.127,0.003, 2.15526]
xh,yh,dim_h = [x2,0.5*x2], [z1, z1+0.5*z2], [0.127,0.003, 2.15526]
xi,yi,dim_i = [x1,0.5*x2], [z1+z2+z3, z1+z2+z3+z4], [0.152,0.0032, 2.15526]
xj,yj,dim_j = [x2,0.5*x2], [z1+z2+z3, z1+z2+z3+z4], [0.152,0.0032, 2.15526]
xk,yk,dim_k = [x1,x2], [z1+z2+z3, z1+z2+z3], [0.102,0.002, 3.048]
xl,yl,dim_l = [x1,x2], [z1+z2, z1+z2], [0.102,0.002, 3.048]
xm,ym,dim_m = [x1,x2], [z1, z1], [0.102,0.002, 3.048]

#sl=4. #sea level
sigy=284 #MPa
el_id=['a','b','c','d','e','f','g','h','i','j','k','l','m']

d=np.array([dim_a,dim_b,dim_c,dim_d,dim_e,dim_f,dim_g,dim_h,
              dim_i,dim_j,dim_k,dim_l,dim_m]).T
dim = pd.DataFrame(d ,index=['diameter','thickness','length'], columns=el_id)

In [ ]:
#Draw frame
#from matplotlib.ticker import AutoMinorLocator
#from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,AnnotationBbox)
#from labellines import labelLine, labelLines
fig1=plt.subplots()
plt.plot(xa,ya,xb,yb,xc,yc,xd,yd,xe,ye,xf,yf,xg,yg,xh,yh,xi,yi,xj,yj,
         xk,yk,xl,yl,xm,ym,marker = '.',color='grey')
plt.axis('equal')

In [ ]:
dim

In [ ]:
index_impacted = ['a','b','c','d','i','j','k','l']

# Computing energy max
Store:
* energy_max
* energy_max_index
* energy_max_samp

In [ ]:
energy_max = np.zeros((2, int(1e5)))

od = dim['a']['diameter']
wt = dim['a']['thickness']

samp = int(1e5)
k = np.random.uniform(1250, 12800, size=(samp))
m = np.random.normal(2.75, 0.12, size=(samp)) 

sigy = 284 #MPa
Np = sigy*np.pi*wt*(od-wt)
Mp = sigy*wt*(od-wt)**2
l = np.sqrt((xa[0]-xa[1])**2 + (ya[0]-ya[1])**2)

thmax = k*(wt/od)**m
Emax = 4*Mp*thmax+0.5*Np*l*(thmax**2)
energy_max[0] = Emax

od = dim['i']['diameter']
wt = dim['i']['thickness']

samp = int(1e5)
k = np.random.uniform(1250, 12800, size=(samp))
m = np.random.normal(2.75, 0.12, size=(samp)) 

sigy = 284 #MPa
Np = sigy*np.pi*wt*(od-wt)
Mp = sigy*wt*(od-wt)**2
l = np.sqrt((xi[0]-xi[1])**2 + (yi[0]-yi[1])**2)

thmax = k*(wt/od)**m
Emax = 4*Mp*thmax+0.5*Np*l*(thmax**2)
energy_max[1] = Emax

energy_max_index = np.array([0,0,0,0,2,2,2,2,1,1,0,0,2],dtype=int)
energy_max_samp = samp

# pf_mc = np.sum(Emax < energy_mc) /samp

In [ ]:
energy_max.shape

In [ ]:
energy_max_index

In [ ]:
energy_max_samp

# Storing input info

In [ ]:
np.savez('collision_info.npz', col_intens=col_intens, energy_max=energy_max, 
         energy_max_index=energy_max_index, energy_max_samp=energy_max_samp)

## Method for  $impact$_$energy$ 

In [ ]:
def energy_collision(impact_energy_start):
    collision_events = np.random.poisson(lam = freq_col, size=None)
    index_impact_braces = np.array([0, 1, 2, 3, 8, 9, 10, 11], dtype = int)
    impact_energy = impact_energy_start.copy()
    for i in range(len(freq_col)):
        if collision_events[i] > 0:
            for _ in range(collision_events[i]):
                ind_energy = np.random.choice(index_impact_braces)
                impact_energy[ind_energy] += col_intens[i]
    return impact_energy

In [ ]:
impact_energy_start = np.zeros(13)
imp_energy = energy_collision(impact_energy_start)
imp_energy

In [ ]:
imp_energy

## Method for computing $p_{f_{col}}$

In [ ]:
def pf_col(energy):
    pf_brace = np.zeros(13)
    for i in range(13):
        if energy[i] > 0:
            pf_brace[i] = np.sum(energy_max[energy_max_index[i]] < energy[i]) / energy_max_samp
    return pf_brace

In [ ]:
pf_col(imp_energy)

## Examples for the calculation of $E_{max}$

### Element 'a'

In [ ]:
od = dim['a']['diameter']
wt = dim['a']['thickness']

samp = int(1e5)
k = np.random.uniform(1250, 12800, size=(samp))
m = np.random.normal(2.75, 0.12, size=(samp)) 

sigy = 284 #MPa
Np = sigy*np.pi*wt*(od-wt)
Mp = sigy*wt*(od-wt)**2
l = np.sqrt((xa[0]-xa[1])**2 + (ya[0]-ya[1])**2)

thmax = k*(wt/od)**m
Emax = 4*Mp*thmax+0.5*Np*l*(thmax**2)

# bin = 0.0125 - 0.125
 
print('prob_IM_1', np.sum(Emax < 0.000625) /samp)
print('prob_IM_2', np.sum(Emax < 0.003125) /samp)
print('prob_IM_3', np.sum(Emax < 0.00625) /samp)
print('prob_IM_4', np.sum(Emax < 0.009375) /samp)
print('prob_IM_5', np.sum(Emax < 0.0125) /samp)

### Element 'i'

In [ ]:
od = dim['i']['diameter']
wt = dim['i']['thickness']

samp = int(1e5)
k = np.random.uniform(1250, 12800, size=(samp))
m = np.random.normal(2.75, 0.12, size=(samp)) 

sigy = 284 #MPa
Np = sigy*np.pi*wt*(od-wt)
Mp = sigy*wt*(od-wt)**2
l = np.sqrt((xi[0]-xi[1])**2 + (yi[0]-yi[1])**2)

thmax = k*(wt/od)**m
Emax = 4*Mp*thmax+0.5*Np*l*(thmax**2)

# bin = 0.0125 - 0.125

print('prob_IM_1', np.sum(Emax < 0.000625) /samp)
print('prob_IM_2', np.sum(Emax < 0.003125) /samp)
print('prob_IM_3', np.sum(Emax < 0.00625) /samp)
print('prob_IM_4', np.sum(Emax < 0.009375) /samp)
print('prob_IM_5', np.sum(Emax < 0.0125) /samp)

# Cells for check

In [ ]:
from scipy.stats import poisson
#N=10000 lifes 
years=30
e=np.empty((len(freq_col),years))
el_imp=[]
en_imp=[]
for j in range (0,years):
    el=[]
    en=[]
    for i in range (0,len(freq_col)):
        e[i,j] = np.random.poisson(lam=freq_col[i], size=None)
        #Element impacted
        for k in range (0,int(e[i,j])):
            el.append(np.random.choice(['a','b','c','d','i','j','k','l']))
            en.append("%.3f" % col_intens[i])       
    el_imp.append(el)
    en_imp.append(en)
#events=pd.DataFrame(e,index=[str(x)+'MJ' for x in Ecol])
#events['total'] = events.sum(axis=1)
elements = pd.DataFrame(np.array([el_imp,en_imp],dtype=object))
elements

In [ ]:
d.shape[1]

In [ ]:
pf=np.zeros((d.shape[1],years))
ecol=np.zeros((d.shape[1],years))
Eav=np.empty((d.shape[1],years))
for j in range (0,years):
    rep=[]
    impacted_elements=elements[j][0]       
    for k in range (0,d.shape[1]):
        rep.append({el_id[k]:impacted_elements.count(el_id[k]) for i in impacted_elements})
    for i in range (0,len(impacted_elements)):      
        ii=el_id.index(elements[j][0][i])
        ecol[ii,j]=elements[j][1][i]
        od,wt=dim[elements[j][0][i]][0],dim[elements[j][0][i]][1]
        print(od,wt)
        l=np.sqrt((xd[0]-xd[1])**2+(yd[0]-yd[1])**2)
        #Fully plastic
        Np=sigy*np.pi*wt*(od-wt)
        Mp=sigy*wt*(od-wt)**2
        ##Average for max angle and energy
        #thmax_av=1250*(wt/od)**2.75
        #Emax_av=4*Mp*thmax_av+0.5*Np*l*(thmax_av**2)       
        n=100000
        Emax=[]
        for x in range (0,n): 
            k=np.random.uniform(1250,12800)
            m=np.random.normal(2.75, 0.12) 
            thmax=k*(wt/od)**m
            Emax.append(4*Mp*thmax+0.5*Np*l*(thmax**2))
        efail=[e for e in Emax if ecol[ii,j]>e]
        pf[ii,j]=len(efail)/len(Emax)        
fail=pd.DataFrame(pf,dtype=object,index=el_id)

In [ ]:
fail

In [ ]:
#Consider previous year damage 
pfcum=np.zeros((d.shape[1],years))
for j in range (0,years):
    el=elements[j][0]
    pfcum[:,0]=pf[:,0] 
    if j>0:
        for k in range (0,d.shape[1]):
            if el_id[k] not in el: 
                pfcum[k,j]=pfcum[k,j-1]
            if el_id[k] in el and (pfcum[k,j-1]==0.0 or pf[k,j-1]==1.0): 
                pfcum[k,j]=pf[k,j]
            if el_id[k] in el and 0.0<pfcum[k,j-1]<1.0:
                ecoltot=np.sum(ecol[k][0:j])
                efail_tot=[e for e in Emax if ecoltot>e]
                pfcum[k,j]=len(efail_tot)/len(Emax)
failcum=pd.DataFrame(pfcum,dtype=object,index=el_id)

In [ ]:
failcum